# Fifteen minutes walking for urban city planning
To create a 15-minute city, urban planners need to address a variety of place categories to ensure that all essential services and amenities are within easy reach.

In [16]:
from arcgis.gis import GIS
from arcgis.features import FeatureSet
from arcgis.geometry.functions import project
from geopedestrian.categories import CategoryRegistry
from geopedestrian.services import analyze_walking
from georapid.client import GeoRapidClient
from georapid.factory import EnvironmentClientFactory

The `host` variable stores the URL for the geopedestrian API hosted on RapidAPI. This URL is used to connect to the GeoPedestrian services.

The `client` variable creates an instance of the `GeoRapidClient` using the `EnvironmentClientFactory`. The `create_client_with_host` method takes the host URL as an argument and returns a client instance for interacting with the geopedestrian API.

In [3]:
host = 'geopedestrian.p.rapidapi.com'
client: GeoRapidClient = EnvironmentClientFactory.create_client_with_host(host)

This function plots the result of a 15-minute walking analysis using the specified map view. The map view must be fully initialized before calling this function.

In [39]:
def plot_urban_walking(map_view, client: GeoRapidClient, category_id: int):
    """
    Plots the 15 minutes walking analysis result using the specified map view.
    The map view must be fully initialized.
    
    :param map_view: The map view defining the area of interest.
    :param client: The authenticated client instance for Rapid API.
    :param category_id: The places category of interest.
    :return: Returns the analysis result as a spatially-enabled dataframe.
    """
    # Project the map view center to WGS84
    spatial_reference = map_view.center['spatialReference']
    in_sr = spatial_reference['wkid']
    out_sr = 4326
    project_result = project([map_view.center], in_sr, out_sr)
    location = project_result[0]
    lat, lon = location.y, location.x
    
    # Analyze the 15 minutes walking areas
    analysis_result = analyze_walking(client, lat, lon, category_id)
    analysis_sdf = FeatureSet.from_geojson(analysis_result).sdf
    analysis_sdf.spatial.plot(map_view, renderer_type='u', col='hits', cmap='YlGn', alpha=0.35)
    map_view.zoom_to_layer(analysis_sdf)
    return analysis_sdf

## Define the categories of interest
This line of code creates an instance of the `CategoryRegistry` class and assigns it to the `category_registry` variable. The `CategoryRegistry` instance can be used to manage and access different categories of places for the 15-minute city analysis.

In [7]:
category_registry = CategoryRegistry()

This line of code searches the `CategoryRegistry` instance for the category ID corresponding to the category name "Museum". The resulting ID is then assigned to the museum_id variable. This ID can be used in subsequent analysis or API calls to identify places related to museums.

In [38]:
museum_id = category_registry.find_id('Museum')

## Visualize the 15 minutes walking scenarios for various cities

In [9]:
gis = GIS()

In [10]:
def create_map_view(gis, named_location):
    map_view = gis.map(named_location)
    map_view.basemap = 'osm'
    return map_view

In [40]:
paris_map = create_map_view(gis, 'Paris, France')
paris_map

MapView(layout=Layout(height='400px', width='100%'))

In [41]:
plot_urban_walking(paris_map, client, museum_id)

,hits,OBJECTID,SHAPE
0,1,1,"{""rings"": [[[2.3549608912799354, 48.8545387117..."
1,1,2,"{""rings"": [[[2.3549608912799354, 48.8607445322..."
2,1,3,"{""rings"": [[[2.3549608912799354, 48.8660631946..."
3,1,4,"{""rings"": [[[2.3561278370649186, 48.8603012848..."
4,1,5,"{""rings"": [[[2.3607956202048506, 48.8567551645..."
5,1,6,"{""rings"": [[[2.3607956202048506, 48.8585282561..."
6,1,7,"{""rings"": [[[2.3619625659898333, 48.8598580335..."
7,1,8,"{""rings"": [[[2.3666303491297653, 48.8589715192..."
